# Radar Data Regridding and Creation

This data is downloaded from https://mesonet.agron.iastate.edu/GIS/rasters.php?rid=4.

In [ ]:
import requests

import xarray as xr
import netCDF4 as nc

import datetime
import numpy as np

import cartopy
import matplotlib.pyplot as plt
import matplotlib as mpl

import xesmf as xe

In [ ]:
def radar_colormap():
    
    
    nws_reflectivity_colors = [
        "#ccffff", # -30
        "#cc99cc", # -25
        "#996699", # -20
        "#663366", # -15
        "#cccc99", # -10
        "#999966", # -5
        "#646464", # 0
        "#04e9e7", # 5
        "#019ff4", # 10
        "#0300f4", # 15
        "#02fd02", # 20
        "#01c501", # 25
        "#008e00", # 30
        "#fdf802", # 35
        "#e5bc00", # 40
        "#fd9500", # 45
        "#fd0000", # 50
        "#d40000", # 55
        "#bc0000", # 60
        "#f800fd", # 65
        "#9854c6", # 70
        "#fdfdfd" # 75
        ]

    return mpl.colors.ListedColormap(nws_reflectivity_colors)

new_cmap = radar_colormap()

In [ ]:
#obtain the file for the MRMS lowest level reflectivity from the Iowa State website.
url = 'https://mesonet.agron.iastate.edu/cgi-bin/request/raster2netcdf.py?dstr=202211042100&prod=mrms_lcref'
url
response = requests.get(url)
open(f'/home/scratch/jcorner1/IA_ST_Radar/mrms_lcref{url[-28:-16]}.nc4', 'wb').write(response.content)


In [ ]:
#open the MRMS and HRRR dataset.
ds_in = xr.open_dataset('/home/scratch/jcorner1/IA_ST_Radar/mrms_lcref202211042100.nc4', engine='netcdf4')
ds_hrrr = xr.open_dataset('hrrr_esrl_2019052800f027.grib2', filter_by_keys={'typeOfLevel': 'atmosphere'})
ds_hrrr


## Regridding the Data
Use the HRRR data to regrid the MRMS data to the proper grid spacing

In [ ]:
#Create the regridder.
regridder = xe.Regridder(ds_in, ds_hrrr, "bilinear")
regridder


In [ ]:
#Regrid the data.
ds_out = regridder(ds_in)
ds_out


In [ ]:
#Save the regridded data.
date_time = datetime.datetime.strptime(url[-28:-16], '%Y%m%d%H%M')
datetime_object = date_time.strftime('%Y-%m-%dT%H:%M:00')

new_ds = ds_in.assign_coords({"time": datetime_object})
new_ds = new_ds.assign_coords(lon=ds_out.longitude)
new_ds = new_ds.assign_coords(lat=ds_out.latitude)
new_ds = new_ds.assign_coords(mrms_lcref=ds_out.mrms_lcref)

#new_ds.to_netcdf(f'/home/scratch/jcorner1/IA_ST_Radar/mrms_lcref{datetime_object}.nc4')
new_ds.to_netcdf(f'mrms_lcref{datetime_object}.nc4')

## Create a Plot for the Regridded Data.

In [ ]:
#Open the regridded dataset.
dataset = xr.open_dataset(f'/home/scratch/jcorner1/IA_ST_Radar/mrms_lcref{datetime_object}.nc4')
dataset

In [ ]:
#Create a mask for low reflectivity data
ds_masked = dataset.where(dataset['mrms_lcref'] != -32.)
ds_masked = ds_masked.where(dataset['mrms_lcref'] != 0.0)


In [ ]:
#Create the figure.
fig = plt.figure()
fig.set_size_inches(16,12)
ax1 = fig.add_subplot(111, projection = cartopy.crs.LambertConformal(central_latitude = 40., central_longitude = -105.))


ax1.add_feature(cartopy.feature.STATES.with_scale('50m'), facecolor='none', zorder = 9)
ax1.add_feature(cartopy.feature.OCEAN)
ax1.add_feature(cartopy.feature.COASTLINE)
ax1.add_feature(cartopy.feature.BORDERS)

ax1.set_extent([-122,-77,20,50])
norm = mpl.colors.Normalize(vmin=-30, vmax=80)
ax1.pcolormesh(ds_masked.longitude.values, ds_masked.latitude.values, ds_masked.mrms_lcref.values, transform=cartopy.crs.PlateCarree(), shading='auto', norm = norm, cmap= new_cmap)